In [52]:
import json
import os
import pandas
import math

# Put the txt files (H1-160.txt) in the input directory
INPUT_PATH = './Data/Data_2/0_Input/'
OUTPUT_PATH = './Data/Data_2/1_AeULDs/Ulds_scenario_1'

In [53]:
def createOutputFolderIfNotExists():
    isExist = os.path.exists(OUTPUT_PATH)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(OUTPUT_PATH)
       print("The new directory is created!")

In [54]:
def readText(filePathImport):
    try:
        return pandas.read_table(filePathImport) 
    except exc:
        print(exc)

In [55]:
def readJson(filePathImport):
    #Read file in
    with open(filePathImport) as json_file:
        json_file = json.load(json_file)
        return json_file

In [56]:
def writeJson(filePathExport, jsonFile):
    with open(filePathExport, 'w', encoding='utf-8') as f:
        json.dump(jsonFile, f, ensure_ascii=False, indent=4)

In [57]:
def createULD(uld_id, items):
    newUld = {"placedItems":[], "id":1}
    #TODO: Put here correct ULD
    newUld["properties"] = readJson("./01_masterdata/0_ulds/pmc_D.json")["properties"]
    sequenceCounter = 0
    
    for item in items:
        item_parsed = readJson('./01_masterdata/1_items/placeholders_placed_item.json')
        
        volume = item[2] * item[3] * item[4]

        shape = {
            'shapeType' : 'Box',
            'width': item[2],
            'height': item[4],
            'depth': item[3],
            'rotationState': 0,
            'id': 2025,
            'volume': volume
        }
        
        item_parsed['shape'] = shape
        item_parsed['sequence'] = sequenceCounter
        item_parsed['x'] = item[5]
        item_parsed['y'] = item[7] # z-axis from Data maps to our y-axis
        item_parsed['z'] = item[6]

        item_parsed['itemLabel'] = uld_id + "_" + str(item_parsed['sequence'])
        item_parsed['shipmentLabel'] = uld_id + "_" + str(item_parsed['sequence'])
        item_parsed['weight'] = round(math.sqrt(volume), 2)
        item_parsed['loadCapacity'] = item_parsed['weight']
        item_parsed['rotationAxes'] = 'XYZ'
    
        newUld['placedItems'].append(item_parsed)
        sequenceCounter = sequenceCounter + 1

    return newUld

In [58]:
def parseULDs(df_grouped):
    ulds = {}
    for index, row in df_grouped.iterrows():
        key = "Class_" + str(index[0]) + "_Instance_" + str(index[1]) + "_ULD_" + str(index[2])
        items = []
        for i in range(len(row.to_list()[0])):
            item = []
            for column in row.to_list():
                item.append(column[i])
            items.append(item)
        ulds[key] = items
    return ulds

In [59]:
## run through flights
def transformFile(sourceFilePath, sourceFile):
    df = readText(sourceFilePath)
    grouped = df.groupby(['CL.No.', 'IN.No.', 'Bin.No']).agg(list)
    
    ulds = parseULDs(grouped)

    for uld_ID, items in ulds.items():
        path_output_full = OUTPUT_PATH + '/' + sourceFile.split('.')[0] + "_" + uld_ID + ".json"
        newUld = createULD(uld_ID, items)       
        writeJson(path_output_full, newUld)

In [60]:
from tqdm import tqdm


def execute():
    createOutputFolderIfNotExists()

    for f in tqdm(os.listdir(INPUT_PATH)):
        transformFile(INPUT_PATH + "/" + f, f)

In [61]:
execute()

The new directory is created!


100%|██████████| 160/160 [00:20<00:00,  7.69it/s]
